In [1]:
import pandas as pd
import numpy as np

mer = pd.read_json("023_JV_segments_matched_speaker_id_added.jsonl", orient="records", lines=True)
mer = mer[~mer.file.str.contains("_BzZf0fGg0E")]
corp = pd.read_json("020_JV_with_metadata.jsonl", orient="records", lines=True)

def corpus_handler(s:str) -> str:
    """For corpus transcripts."""
    from string import punctuation
    for p in punctuation:
        s = s.replace(p, "")
    return " ".join(
        s.replace("JV:", "").split()
    ).casefold()

def segment_handler(s:str) -> str:
    return s.replace("<anchor_start>", "").replace("<anchor_end>", "")

def find_optimal_subset(segment, full, step=1, limit = None):
    from fuzzywuzzy.fuzz import ratio
    from tqdm.auto import tqdm
    segment = segment.split()
    full = full.split()
    best = 0
    best_start, best_end = 0, -1
    tqdm = lambda x: x
    for start in tqdm(range(0, len(full) if not limit else 2*limit, step)):
        for end in range(len(full)- 2*limit if limit else start, len(full), step):
            try:
                subset = full[start:end]
                current = ratio(
                    segment_handler(" ".join(segment)),
                    corpus_handler(" ".join(subset))
                        )
                # print(start, end, current, subset)
                if current >= best:
                    best_start, best_end = start, end
                    best = current
            except IndexError:
                continue
    if step != 1:
        best_start = max((best_start - step, 0))
        best_end   = min((best_end   + step, len(full)))
    return " ".join(
        full[best_start:best_end]
    )

In [2]:
i = 300
segments_file = mer.file.values[i]
full_transcript = corp[corp.path == segments_file].transcript.values[0]
segments_kaldi_transcript = mer.kaldi_transcript.values[i]

It has been found out that the best course of action is to set up a step-down strategy: instead of matching words to words, it is best to do it first on hecto-words and then only identify word level matches when the best candidate string is far shorter.

Specifically, when doing 100->1 step down, the execution time drops from 5h to 4s, which is a speed-up of more than 36 dB!

Since this was not fast enough, I also implemented a three step version, and the algorithm has been corrected so that after first iteration we only trim the ends of the string.

In [3]:
coarse = find_optimal_subset(segments_kaldi_transcript, full_transcript, step=100)
medium = find_optimal_subset(segments_kaldi_transcript,  coarse, step=10, limit=100)
fine = find_optimal_subset(segments_kaldi_transcript,  medium, step=1, limit=10)

In [4]:
fine

'otklonili te probleme? 364 zdravstvene ustanove postoje u planu mreže u Srbiji. 14 ih ima u Nišu. Ukoliko kupujete 36 olovke, to je po novom zakonu, biće jeftinije nego ako kupujete 1. Isto tako se pokazalo se, pokazalo se i par problema, o kojima ja sada govorim,'

In [5]:
segments_kaldi_transcript

'otklonili te probleme<anchor_end> tristo šezdeset četiri zdravstvene ustanove postoje u planu mreže u srbiji četrnaest ih ima u nišu ukoliko kupujete trideset šest olovke to je po novom zakonu biće jeftinije nego ako kupujete jedan isto tako se pokazalo se pokazalo se i par problema o kojima ja sada govorim'

# running it full scale

In [6]:



def match_kaldi(row):
    try:
        segments_file = row["file"]
        full_transcript = corp[corp.path == segments_file].transcript.values[0]
        segments_transcript = row["kaldi_transcript"]

        coarse = find_optimal_subset(segments_transcript, full_transcript, step=100)
        medium = find_optimal_subset(segments_transcript,  coarse, step=10, limit=100)
        fine = find_optimal_subset(segments_transcript,  medium, step=1, limit=10)
        return fine
    except:
        return ""

from concurrent.futures import ProcessPoolExecutor

with ProcessPoolExecutor(max_workers=32) as executor:
    results = executor.map(match_kaldi, [row for i, row in mer.iterrows()])
mer["raw_transcript__matched_on_kaldi"] = list(results)

In [7]:
mer.to_json("025_segments_matched_with_raw.jsonl", orient="records", lines=True)

In [8]:
def match_asr(row):
    try:
        segments_file = row["file"]
        full_transcript = corp[corp.path == segments_file].transcript.values[0]
        segments_transcript = row["asr_transcription"]

        coarse = find_optimal_subset(segments_transcript, full_transcript, step=100)
        medium = find_optimal_subset(segments_transcript,  coarse, step=10, limit=100)
        fine = find_optimal_subset(segments_transcript,  medium, step=1, limit=10)
        return fine
    except:
        return ""


from concurrent.futures import ProcessPoolExecutor
with ProcessPoolExecutor(max_workers=32) as executor:
    results = executor.map(match_asr, [row for i, row in mer.iterrows()])
mer["raw_transcript__matched_on_asr"] = list(results)

In [9]:
mer.to_json("025_segments_matched_with_raw.jsonl", orient="records", lines=True)

In [10]:
import pandas as pd

mer = pd.read_json("025_segments_matched_with_raw.jsonl", orient="records", lines=True)

In [11]:
mer.columns

Index(['file', 'start', 'end', 'asr_transcription', 'kaldi_transcript',
       'guest_name', 'guest_description', 'host', 'kaldi_words',
       'kaldi_word_starts', 'kaldi_word_ends', 'segment_file',
       'average_distance', 'similarity_ratio', 'speaker_breakdown',
       'Raw_transcript__matched_on_kaldi', 'Raw_transcript__matched_on_asr'],
      dtype='object')

In [13]:
mer[[
'file', 
'segment_file',
'start', 
'end', 
'asr_transcription', 
'kaldi_transcript',
'raw_transcript__matched_on_kaldi', 
'raw_transcript__matched_on_asr',
'guest_name', 
'guest_description', 
'host', 
'kaldi_words',
'kaldi_word_starts',
'kaldi_word_ends', 
'speaker_breakdown',
'average_distance',
'similarity_ratio',
]].to_json("025_segments_matched_with_raw.jsonl", orient="records", lines=True)

In [ ]:
pd.set_option("display.max_colwidth", 100)
mer.sort_values("similarity_ratio")["asr_transcription kaldi_transcript Raw_transcript__matched_on_kaldi Raw_transcript__matched_on_asr".split()]

,asr_transcription,kaldi_transcript,Raw_transcript__matched_on_kaldi,Raw_transcript__matched_on_asr
3418,moje generacija je odrastala slušajući bajki koji je pričao zijaksokolović palčić trno lužica zl...,moje ime je,broj i misle da će,"generacija je odrastala slušajući bajke koje je pričao Zijah Sokolović, Palčić, Trnoružica, Zlat..."
11043,folk svezda svetlana cecera žnatović nastupit će za srpsku novu godinu unišu kakvu poruku ovim v...,folk ne znam da,znao da,Folk zvezda Svetlana Ceca Ražnjatović nastupiće za srpsku novu godinu u Nišu. Kakvu poruku ovim ...
11461,koliko će još dugo deponija bubanj biti u funkciji kako će se otpad u budućnosti odlagati i šta ...,koliko je ljudi upozoreno i koliko je,"ostali mediji, imate sudski proces koji je",Kako će se otpad u budućnosti odlagati i šta će biti sa vanpijačnom prodajom u Nišu? Moje ime je...
8915,naš današnji gos bio je u svim vladama poslije 20. godine ni u jedno i nije ostop po mandat mlađ...,naš gost moje ime je predrag blagojević bila je,osam od deset pitanja je bilo šta je,"današnji gost bio je u svim vladama od 2000. godine i ni u jednoj nije ostao pun mandat, Mlađan ..."
12449,srbiji odavno nije u republitičkom parlamentu predstavljao neko tko koga bar građani ne poznaju ...,srbiji već pita gde će dete da mi ide u,vratiti i da vreme radi samo za Demokratsku,"Srbije odavno nije u republičkom parlamentu predstavljao neko koga, bar građani, ne poznaju samo..."
...,...,...,...,...
11365,mi smo pokušali da preko strukovnih udruženja ukažemo na ovaj problem sva reprezentativna udruže...,mi smo pokušali da preko strukovnih udruženja ukažemo na ovaj problem sva reprezentativna udruže...,Mi smo pokušali da preko strukovnih udruženja ukažemo na ovaj problem. Sva reprezentativna udruž...,Mi smo pokušali da preko strukovnih udruženja ukažemo na ovaj problem. Sva reprezentativna udruž...
349,mnogo sam godina bio novinar aktivno radio i znam kako je bilo u različitim vremenima,mnogo sam godina bio novinar aktivno radio i znam kako je bilo u različitim vremenima,"Mnogo sam godina bio novinar, aktivno radio i znam kako je bilo u različitim vremenima.","Mnogo sam godina bio novinar, aktivno radio i znam kako je bilo u različitim vremenima."
353,redakcija je uglavnom stajala iza svojih novinara i to je nešto što je jako bitno danas mislim d...,redakcija je uglavnom stajala iza svojih novinara i to je nešto što je jako bitno danas mislim d...,"pratim, jednostavno sklapam mozaik iz informacija koje čujem. To nije dobro ali današnje vreme j...","pratim, jednostavno sklapam mozaik iz informacija koje čujem. To nije dobro ali današnje vreme j..."
172,šta je konkretno bio vaš motiv,šta je konkretno bio vaš motiv<anchor_end>,da će možda neko od tih 46 vaših kolegai,da će možda neko od tih 46 vaših kolegai


In [ ]:
mer.Raw_transcript__matched_on_asr.values[8]

'je, da kažem, jedna ustanova. Igrom slucaja je dr Radovanović na čelu te ustanove. Tako je. JV: Vi ste donedavno bili i direktor jednog lokalnog javnog preduzeća. Ali ono što vas pitam jeste, prvenstveno, šta ste vi uradili ili planirate da uradite kao narodni poslanici?'

In [ ]:
mer.head()

,file,start,end,asr_transcription,kaldi_transcript,guest_name,guest_description,host,kaldi_words,kaldi_word_starts,kaldi_word_ends,segment_file,average_distance,similarity_ratio,Raw_transcript__matched_on_kaldi,Raw_transcript__matched_on_asr
0,audio/58xZSVbpgkk.wav,12.0,31.0,nekoliko puta smo ponavljali da nišo od uvođenja više stranačja nije ni kad imao veći broj narod...,nekoliko tema je bilo u skupštini znači niške teme su bile <anchor_start>dobro ne moramo o ljudi...,Slavoljub Vlajković,narodni poslanik SPS iz Niša,Predrag Blagojević,"[nekoliko, tema, je, bilo, u, skupštini, znači, niške, teme, su, bile, <anchor_start>dobro, ne, ...","[12.47, 13.3, 13.45, 13.51, 13.65, 14.66, 15.0, 15.56, 16.32, 16.5, 16.62, 16.79, 17.02, 18.16, ...","[12.89, 13.45, 13.51, 13.65, 13.68, 15.0, 15.56, 15.94, 16.5, 16.62, 16.79, 17.02, 17.16, 18.37,...",seg_audio/58xZSVbpgkk__12.0__31.0__.wav,5.142857,44,"nekoliko tema je bilo u Skupštini! Znači, niške teme su bile... JV: Dobro, ne moramo o ljudima! ...",Nekoliko puta smo ponavljali da Niš od uvođenja višestranačja nijenikada imao veći broj poslanik...
1,audio/58xZSVbpgkk.wav,31.0,47.0,nikad više pa niti iskoristio niti će izgledati iskoristi tu priliku što mislim da je to pa prob...,nikad više<anchor_end> niti je iskoristio niti će izgleda iskoristi priliku mislim da je to <anc...,Slavoljub Vlajković,narodni poslanik SPS iz Niša,Predrag Blagojević,"[nikad, više<anchor_end>, niti, je, iskoristio, niti, će, izgleda, iskoristi, priliku, mislim, d...","[31.17, 31.46, 32.62, 32.83, 32.89, 33.54, 33.75, 33.9, 34.22, 35.03, 36.21, 36.48, 36.55, 36.61...","[31.46, 31.74, 32.83, 32.89, 33.54, 33.75, 33.9, 34.22, 35.03, 35.53, 36.48, 36.55, 36.61, 36.67...",seg_audio/58xZSVbpgkk__31.0__47.0__.wav,3.604167,89,"nikad više?’ Niti je iskoristio, niti će izgleda iskoristi priliku. Mislim da je to.. JV: Šta je...","nikad više?’ Niti je iskoristio, niti će izgleda iskoristi priliku. Mislim da je to.. JV: Šta je..."
2,audio/58xZSVbpgkk.wav,47.0,62.5,dobrani i znate da e svaka stranka od polih predloži 200 kandidata slučaj tako te o da smo svi b...,vi znate da svaka stranka otprilike predloži dvijesto pedeset kandidata slučaj je tako hteo da s...,Slavoljub Vlajković,narodni poslanik SPS iz Niša,Predrag Blagojević,"[vi, znate, da, svaka, stranka, otprilike, predloži, dvijesto, pedeset, kandidata, slučaj, je, t...","[47.74, 47.84, 48.12, 48.19, 48.49, 48.89, 49.4, 49.72, 50.1, 50.45, 51.67, 52.12, 52.18, 52.54,...","[47.84, 48.12, 48.19, 48.49, 48.89, 49.22, 49.72, 50.07, 50.45, 51.26, 52.12, 52.18, 52.49, 52.7...",seg_audio/58xZSVbpgkk__47.0__62.5__.wav,5.729730,89,znate da svaka stranka otprilike predloži 250 kandidata . Slučaj je tako hteo da smo svi bili na...,izabrani. Vi znate da svaka stranka otprilike predloži 250 kandidata . Slučaj je tako hteo da sm...
3,audio/58xZSVbpgkk.wav,62.5,82.1,znači grad uopštenije organizvan i mi faktički u gradu nemamo i jednu jedinu zajedničku želju ko...,znači grad uopšte nije organizovan i mi faktički u gradu nemamo ni jednu jedinu zajedničku želju...,Slavoljub Vlajković,narodni poslanik SPS iz Niša,Predrag Blagojević,"[znači, grad, uopšte, nije, organizovan, i, mi, faktički, u, gradu, nemamo, ni, jednu, jedinu, z...","[62.98, 63.28, 63.64, 63.96, 64.14, 64.91, 64.99, 65.25, 65.88, 66.05, 66.36, 66.78, 67.13, 67.5...","[63.28, 63.64, 63.96, 64.14, 64.88, 64.99, 65.22, 65.88, 66.05, 66.36, 66.78, 66.84, 67.55, 67.9...",seg_audio/58xZSVbpgkk__62.5__82.1__.wav,4.886364,94,"Znači, grad uopšte nije organizovan i mi faktički u gradu nemamo ni jednu jedinu zajedničku želj...","Znači, grad uopšte nije organizovan i mi faktički u gradu nemamo ni jednu jedinu zajedničku želj..."
4,audio/58xZSVbpgkk.wav,82.1,98.4,pa iima ali ja kažem da bi trebao grada kažem is to ja mogu individualno tako e socijalistička p...,pa ima ali ja kažem da bi trebalo grad da kaže nisu to ja mogu individualno ali tako mogu u ime ...,Slavoljub Vlajković,narodni poslanik SPS iz Niša,Predrag Blagojević,"[pa, 